# Introdução a Pandas para Ciência de Dados - Conceitos intermediários

Neste notebook apresentamos outros conceitos importantes do Pandas. 


## Inicializando DataFrames a partir de dicionários

Uma das formas mais fácies de se inicializar um DataFrame é usando dicionários. Uma lista de dicionários pode ser usada para compor as linhas do DataFrame como no exemplo abaixo:

In [ ]:
import pandas as pd

ap1 = {'endereco': 'Av V. Guarapuava, 1000', 'area': 45, 'aluguel': 800}
ap2 = {'endereco': 'Av Sete de Setembro, 170', 'area': 53, 'aluguel': 950}

apartamentos = [ap1, ap2]

df = pd.DataFrame(apartamentos)
df

Podemos usar um dicionário para adicionar uma nova linha a um DataFrame:

In [ ]:
ap3 = {'endereco': 'Av Sete de Setembro, 830', 'area': 35, 'aluguel': 850}

df = df.append([ap3])
df

In [ ]:
# usando o endereço como índice para os próximos exemplos
df = df.set_index('endereco')
df

## Iterando sobre um DataFrame

Podemos usar o comando `for` para analisar um DataFrame linha por linha. Para isto usamos o método `iterrows`. O exemplo abaixo calcula a média dos valores dos aluguéis:

In [ ]:
total = 0
for indice, apartamento in df.iterrows():
    print("Aluguel {}: {}".format(indice, apartamento['aluguel']))
    total = total + apartamento['aluguel']
    
media = total/len(df)
print("Média:", media)

## Filtrando linhas com o método query()

Uma forma conveniente de filtrar linhas é usando o método query. No exemplo abaixo selecionamos apenas as linhas com área maior que 40 e aluguel menor que 900.

In [ ]:
df.query("area > 40 and aluguel < 900")

O comando acima é equivalente a:

In [ ]:
df[(df['area'] > 40) & (df['aluguel'] < 900)]

## Blocos básicos de um DataFrame

Um bloco básico na construção de um DataFrame é a Série (Series). Cada coluna de um DataFrame é uma Series. Uma Series contém um nome, um índice e uma lista (array) de valores. Veja os exemplos abaixo baseados na coluna *aluguel*:

In [ ]:
coluna_aluguel = df['aluguel']

# Verificando o tipo da coluna
type(coluna_aluguel)

Índice:

In [ ]:
coluna_aluguel_index = coluna_aluguel.index

# Mostrando o tipo e valores do índice
print(type(coluna_aluguel_index))
print(coluna_aluguel_index)

Valores:

In [ ]:
coluna_aluguel_valores = coluna_aluguel.values

# Mostrando o tipo e valores do índice
print(type(coluna_aluguel_valores))
print(coluna_aluguel_valores)

Como pôde ser visto no exemplo acima, o tipo da coluna aluguéis é `numpy.ndarray`. O Numpy é uma bilioteca de estruturas de dados e operações matemáticas. Abaixo importamos o pacote numpy e usamos uma de suas funções nos valores da coluna:

In [ ]:
import numpy as np

np.mean(coluna_aluguel_valores)

In [ ]:
# Reiniciando índice e adicionando valores para os próximos exemplos
df = df.reset_index()
df = df.append([{'endereco': 'Av Sete de Setembro, 730', 'area': np.NaN, 'aluguel': 775}]).reset_index(drop=True)
df

## Método apply(), eixos de um DataFrame, função *lambda*

O método `apply()` aplica uma função a colunas ou linhas de um DataFrame. No exemplo abaixo definimos uma função que conta o número de valores nulos em uma coluna. Esta função é então aplicada no DataFrame exibido na célula anterior.

In [ ]:
def conta_nan(coluna):
    return coluna.isna().sum()

df.apply(conta_nan)

Uma função também pode ser aplicada sobre valores das linhas de um DataFrame. No caso abaixo, definimos a função `aumenta_aluguel()` e a aplicamos às linhas do DataFrame para criar uma nova coluna com um valor maior para o aluguel. Para especificar que precisamos aplicar a função às linhas, definimos o parâmetro `axis=1`.

In [ ]:
def aumenta_aluguel(linha):
    return linha['aluguel'] + linha['aluguel']*0.1

df['novo aluguel'] = df.apply(aumenta_aluguel, axis=1)

df

O Python possui o conceito de "função anônima", ou "função lambda". Este recurso é útil para especificar um função simples sem precisar defini-la. Abaixo construímos uma função lambda que retorna apenas a parte do número do endereço de um apartamento.

In [ ]:
df['numero ap.'] = df.apply(lambda x: x['endereco'].split(',')[1], axis=1)
df

O código acima é equivalente à sequência abaixo:

In [ ]:
def obtem_numero(linha):
    numero = linha['endereco'].split(',')[1]
    return numero

df['numero ap.'] = df.apply(obtem_numero, axis=1)
df

## Tratando DataFrames grandes

Muitas vezes precisamos processar dados que não cabem na memória do computador ou que demandam procedimentos complexos que deixam o processamento lento. Descrevemos aqui algumas técnicas para amenisar este tipo de problema. Vamos utilizar o dataset de reclamações, inicialmente contendo cerca de 7000 linhas (que é uma quantidade pequena, mas suficiente para os exemplos).

In [ ]:
# lê o arquivo CSV
datafile = '../data/2017-02-01_156_-_Base_de_Dados_sample.csv'
df = pd.read_csv(datafile, sep=';', encoding='latin-1')

print("Total de linhas: ", len(df))

df.head()

Uma forma de reduzir o tamanho de um DataFrame é fazer uma amostragem das linhas. Abaixo fazemos uma amostragem aleatória de 1000 linhas:

In [ ]:
df_sample = df.sample(1000)

print(len(df_sample))

O problema da estratégia de amostragem acima é que os dados já estavam na memória. Portanto, esta estratégia não funcionaria se os dados fossem maior que a memória disponível. Nestes casos precisamos fazer a amostragem no momento da leitura do arquivo de dados.

O código abaixo contrói uma lista aleatória de índices de linhas para serem ignoradas. Quando o comando `pd.read_csv()` é chamado, ele carrega na memória somente as linhas que não aparecem na lista gerada (que foi passada no parâmetro *skiprows*).

In [ ]:
import random

# Definindo uma semente de geração de números aleatórios para que a seleção seja a mesma em múltiplas execuções do código
random.seed(42)

# Conta linhas do arquivo de entrada
num_linhas = sum(1 for l in open(datafile, encoding='latin-1'))

# Define a proporção dos dados a se manter
proporcao = 0.1

# calcula o tamanho desejado da amostragem
novo_tamanho = int(num_linhas * proporcao)

# define os valores de índice aleatórios que serão ignorados
skip_idx = random.sample(range(1, num_linhas), num_linhas - novo_tamanho)

# Lê os dados pulando as linhas definidas
df = pd.read_csv(datafile, sep=';', skiprows=skip_idx, encoding='latin-1')

print("Total de linhas: ", len(df))

Outra possibilidade é ler os dados em "lotes", processando um número determinado de linhas de cada vez. O código abaixo define o tamanho dos pedaços a serem lidos (no caso 30 linhas). Estes pedaços são lidos um de cada vez e cada um recebe um tratamento de limpeza de dados (linhas com valores nulos são eliminadas). Ao fim, apenas cerca de 3600 linhas restaram no DataFrame contruído pelo procedimento.

In [ ]:
chunksize = 30

df_limpa = pd.DataFrame()

for df_chunk in pd.read_csv(datafile, chunksize=chunksize, sep=';', encoding='latin-1'):
    # limpeza de dados no chunk atual:
    df_chunk = df_chunk.dropna(how='any', axis=0)
    df_limpa = pd.concat([df_limpa, df_chunk])

    
print(len(df_limpa))
df_limpa.head()